In [30]:
from pymatgen.io.espresso.PWscf import PWxml
from pymatgen.io.vasp.outputs import vasprun
import xml.etree.ElementTree as ET
from monty.io import zopen
import xmltodict

In [51]:
filename = 'pymatgen/io/espresso/tests/data/Sr3PbO_vcrelax.xml'

In [52]:
PWxml(filename)

xml_format
creator
created
job
general_info
nprocs
nthreads
ntasks
nbgrp
npool
ndiag
parallel_info
title
calculation


AttributeError: 'PWxml' object has no attribute 'parameters'

In [53]:
with open(filename, encoding="UTF-8") as xml:
    data = xmltodict.parse(xml.read())#['qes:espresso']

In [69]:
data['qes:espresso']['output']['band_structure']['ks_energies']

[{'k_point': {'@weight': '3.644314868805e-4',
   '#text': '0.000000000000000e0 0.000000000000000e0 0.000000000000000e0'},
  'npw': '7249',
  'eigenvalues': {'@size': '60',
   '#text': '-9.731827352502439e-1 -9.731827352502416e-1 -9.691671297543149e-1 -9.691671297543138e-1 -9.691671297543131e-1\n          -9.691671297543124e-1 -3.456875637206584e-1 -3.456875637206578e-1 -3.456875637206576e-1 -3.456875637206569e-1\n          -3.334622052078866e-1 -3.334622052078859e-1 -3.237084857493348e-1 -3.237084857493342e-1 -3.099680033600316e-1\n          -3.099680033600309e-1 -3.031547659824096e-1 -3.031547659824092e-1 -3.031547659824090e-1 -3.031547659824075e-1\n          -2.965782503300330e-1 -2.965782503300325e-1 -2.891389134982108e-1 -2.891389134982094e-1 -2.891389134982072e-1\n          -2.891389134982052e-1 -2.878592780795591e-1 -2.878592780795585e-1 -2.878592780795582e-1 -2.878592780795573e-1\n          -1.941120730340013e-1 -1.941120730340008e-1 -1.938943161743397e-1 -1.938943161743397e-1 -